# Zadania do wykonania

1. Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu `pickle`, `parquet` i `xlsx`.

2. Zbadaj przyspieszenie związane z zastosowaniem pamięci podręcznej na wybranych przykładzie funkcji rekurencyjnej (np. ciągu _Fibonacciego_.

3. Napisz program tworzący bazę danych z interfejsem konsolowym. Wymagane są następujące operacje, dodanie wiersza, usunięcie wiersza, zmiana pola wiersza, wyświetlenie opcji. Menu można zorganizować jako odczytywanie parametrów zwróconych przez funkcję `input` lub z pomocą komend (łatwiejszy sposób).

In [89]:
# zadanie 2

import time
from functools import lru_cache, cached_property


# Recursive algorithm for fibbonacci
def fib(n):
    if n <= 1:
        return n
    else:
        return(fib(n-1) + fib(n-2))
        
        
@lru_cache(maxsize=100)
def func_lru_cached(a: int):
    if a <= 1:
        return a
    else:
        return(func_lru_cached(a-1) + func_lru_cached(a-2))        
       

def exec(number):
    start = time.time()
    fib(number)
    end = time.time()
    result = end - start
    print("Time fib normal: ",result,"s")

    start2 = time.time()
    func_lru_cached(number-1)
    end2 = time.time()
    result2 = end2 - start2
    print("Time fib cache: ",end2 - start2,"s")

    print(f'Cache is {result/result2} times faster\n\n')

    
 
print("For 10th element:")
exec(10)

print("For 20th element:")
exec(20)

print("For 30th element:")
exec(30)

print("For 34th element:")
exec(34)

For 10th element:
Time fib normal:  4.100799560546875e-05 s
Time fib cache:  1.52587890625e-05 s
Cache is 2.6875 times faster


For 20th element:
Time fib normal:  0.010749101638793945 s
Time fib cache:  1.5497207641601562e-05 s
Cache is 693.6153846153846 times faster


For 30th element:
Time fib normal:  0.6121826171875 s
Time fib cache:  2.2411346435546875e-05 s
Cache is 27315.744680851065 times faster


For 34th element:
Time fib normal:  3.995593786239624 s
Time fib cache:  8.106231689453125e-06 s
Cache is 492903.9705882353 times faster


